# RNN

RNN利用隐藏层能够捕获之前所有时刻发生的信息

参考：《Python深度学习基于PyTroch》

## 1 Demo实现

![](./pics/RNN_demo.png)

In [1]:
import numpy as np

In [12]:
X = [1, 2]
state = [0.0, 0.0]

In [13]:
w_cell_state = np.array([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
b_cell = np.array([0.1, -0.1])
w_output = np.array([1.0, 2.0])
b_output = 0.1

In [14]:
for i in range(len(X)):
    state = np.append(state, X[i]) # 1x3
    before_activation = np.dot(state, w_cell_state) + b_cell # 1x2
    state = np.tanh(before_activation) # 1X2
    final_output = np.dot(state, w_output) + b_output # 1X1
    print('状态值_%i' % i, state)
    print('输出值_%i' % i, final_output)

状态值_0 [0.53704957 0.46211716]
输出值_0 1.561283881518055
状态值_1 [0.85973818 0.88366641]
输出值_1 2.727071008233731


## 2 Pytorch实现

![](./pics/RNN_model.png)

In [42]:
import torch
import torch.nn as nn

In [43]:
# torch.nn.RNN # pytorch中的RNN使用方式
rnn = nn.RNN(input_size=10, hidden_size=20, num_layers=2)

In [44]:
# 第一层相关参数形状
# ih_l0是输入（10）到隐藏层状态（20），所以是10x20
# hh_l0是状态（20）到状态（20），所以是20x20
rnn.weight_ih_l0.shape, rnn.weight_hh_l0.shape, rnn.bias_hh_l0.shape

(torch.Size([20, 10]), torch.Size([20, 20]), torch.Size([20]))

In [45]:
# 第二层相关参数形状
rnn.weight_ih_l1.shape, rnn.weight_hh_l1.shape, rnn.bias_hh_l1.shape

(torch.Size([20, 20]), torch.Size([20, 20]), torch.Size([20]))

$h_t = \tanh(W_{ih} x_t + b_{ih} + W_{hh} h_{(t-1)} + b_{hh})$

In [36]:
# 生成输入数据
input = torch.randn(100, 32, 10)
h_0 = torch.randn(2, 32, 20)

In [37]:
output, h_n = rnn(input, h_0)

In [34]:
output.shape, h_n.shape

(torch.Size([100, 32, 20]), torch.Size([2, 32, 20]))

In [46]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input_data, hidden):
        combined = torch.cat((input_data, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [17]:
n_hidden = 128